# CCTV 데이터 시각화 

- 과제 제출 기한: 2022.6.21(화) 자정
- 과제 제출 방법
  - 아래 과업을 해결한 주피터 노트북을 'cctv.ipynb' 파일로 저장하여 제출
  - `Kernel > Restart & Run All` 메뉴를 반드시 실행한 후 저장하여 제출
- 과제에서 완성할 시각화 내용
  - 버블차트
    ![bbchart_cctv](https://user-images.githubusercontent.com/10287629/172086730-372009c9-6cd9-4336-beb6-5a9cc8bf64fa.png)
  - 단계구분도
    ![choropleth_cctv](https://user-images.githubusercontent.com/10287629/172055607-be2cff10-7f82-4ee1-9f2e-ada86f3e7efd.png)

- 채점 주안점

![스크린샷 2022-06-06 12 43 50](https://user-images.githubusercontent.com/10287629/172091039-e70d23ee-ec8f-4783-bee4-95fe9354480b.png)

# CCTV 데이터 시각화 



![한양여대 로고](https://www.hywoman.ac.kr/resources/domain/ko/DM_IMG/logo.png)


| 소속     | 한양여자대학교 |   
|----------|----------------|
| 이름     | 윤혜린         |  
| 학번     | 2005943        |   
| 제출일자 | 2022.6.21      |  
                                                                                                  

# 목차

1. 서론
    - What 
    - Why
    - How 
2. 본론
    1. 라이브러리 삽입
    2. 데이터 적재 및 구조 파악
    3. 버블차트 시각화
    4. 단계 구분도 시각화
    5. 결론


## 서론

### What

 cctv_with_pop 데이터와 자치구청 데이터, 서울시 지도 데이터를 이용하여 자치구대 연도별 cctv 증가 비율과 이동궤적, 그리고 자치구별 증가율을 단계구분도로 나타내고자 한다.

### Why

 cctv 설치 증가율을 연도별, 자치구별로 구분해 한 눈에 시각 자료로 시각화하고 이 시각화에 대한 유의미한 결과를 도출해보고 이러한 결과의 원인을 분석해보고자 한다. 더 나아가 이러한 결과를 기반으로 미래 동향도 예측해 보기 위해 이 과제를 수행한다. 

### How

 데이터로 cctv_with_pop 데이터와 서울시 자치구청 데이터, 서울시 지도 데이터를 이용한다.
가상환경에서 주피터 노트북을 이용하여 mark_bubble, transform_regression, 지도 시각화 기술을 통해 구현한다.

## 1. 라이브러리 임포트

In [1]:
# 라이브러리 임포트
import pandas as pd
import altair as alt

## 2. 데이터 적재

- 필요한 데이터는 다음과 같다. 
  - 인구 및 cctv 데이터는 [여기](https://raw.githubusercontent.com/logistex/vd21/main/cctv_with_pop.csv)에서 직접 읽어서 `df`라는 데이터 프레임으로 적재하라.  
    이 데이터에 포함된 `no` 열은 지도 시각화를 위한 [자치구 경계선 데이터](https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json)에 포함되어 있는 자치구의 고유 식별 번호이다. 
  
  - 자치구청 위치 데이터는 [여기](https://raw.githubusercontent.com/logistex/vd21/main/offices.csv)에서 직접 읽어서 `offices`라는 데이터 프레임으로 적재하라. 
  - 자치구 경계선에 관한 지도 데이터는 [여기](https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json)에서 제공되는데, 이에 관한 코드는 (여러분의 수고를 줄여주기 위하여) 문제에 포함시켜 두었다.  
  - 일부 csv 파일은 인코딩 방식을 지정해야만 한글을 제대로 읽을 수 있다. 
  - 데이터 파일은 모두 앞서 제공한 URL 주소로부터 직접 읽어 들이라. 
  - 데이터 프레임으로 적재한 직후에는 임의로 5행을 출력하라. 

In [2]:
# cctv 및 인구 데이터 적재 및 확인
df = pd.read_csv('https://raw.githubusercontent.com/logistex/vd21/main/cctv_with_pop.csv')
df.sample(5)

,no,id,자치구,CCTV_규모_2015,CCTV_규모_2020,인구_규모_2015,인구_규모_2020
16,5,17,구로구,1872,4075,454604,432488
23,19,24,송파구,520,2743,667480,673926
0,6,1,종로구,1175,1729,163822,158996
24,12,25,강동구,725,2547,463321,463998
12,21,13,서대문구,1235,2939,323105,322952


In [3]:
# 지도 시각화를 위한 자치구 위치 데이터 적재 및 확인
offices = pd.read_csv('https://raw.githubusercontent.com/logistex/vd21/main/offices.csv')
offices.sample(5)

,자치구,자치구청,위도,경도,주소
15,강서구,강서구청,37.550910,126.849574,대한민국 서울특별시 강서구 화곡동 화곡로 302
2,용산구,용산구청,37.532590,126.990043,대한민국 서울특별시 용산구 이태원동 녹사평대로 150
6,중랑구,중랑구청,37.606560,127.092624,대한민국 서울특별시 중랑구 신내동 662
11,은평구,은평구청,37.602750,126.929239,대한민국 서울특별시 은평구 녹번동 은평로 195
22,강남구,강남구청,37.517236,127.047325,대한민국 서울특별시 강남구 426


In [4]:
# 자치구 경계선에 관한 지도 데이터
seoul_url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
seoul = alt.topo_feature(seoul_url, 'seoul_municipalities_geo')
seoul

UrlData({
  format: TopoDataFormat({
    feature: 'seoul_municipalities_geo',
    type: 'topojson'
  }),
  url: 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
})

## 3. 버블차트 시각화

### 3.1 회귀선을 포함한 버블차트를 반환하는 함수 정의 

- 함수 `bubble(year, color)`를 정의하라. 
- 매개변수 `year`에 정수 `2015`를 지정하면 2015년 차트를 반환한다. 
- 매개변수 `color`에 색상 이름을 지정하면, 해당 색상으로 버블과 회귀선이 그려진 차트를 반환한다. 


In [5]:
# 회귀선 버블차트 시각화 함수 정의

def bubble(year, color):
    bb = alt.Chart(df).mark_circle(opacity=0.5, color=color).transform_calculate(
        rate='datum.CCTV_규모_'+year+'/datum.인구_규모_'+year
    ).encode(
        alt.X('인구_규모_'+year+':Q',
              scale=alt.Scale(type='linear',domain=[100000,700000]),
              title='인구 규모(단위:명)'),
        alt.Y('CCTV_규모_'+year+':Q',
              scale=alt.Scale(type='linear', domain=[0,7000]), #Y축 설정
              title='CCTV 설치 규모[단위:대]'),
        size=alt.Size('rate:Q', scale=alt.Scale(range=[10,1000]),legend=None), #설정한 버블 크기 적용
        order=alt.Order('rate:Q', sort='descending'), #작은 버블이 위로 가게
        tooltip=[ alt.Tooltip('자치구:N'),
                  alt.Tooltip('rate:Q',format='.1%',title='CCTV 비율'),
                  alt.Tooltip('CCTV_규모_'+year+':Q', title='CCTV 규모'),
                  alt.Tooltip('인구_규모_'+year+':Q', title='인구 규모')]#툴팁
      ).properties(
            title={
                "text":["자치구별 인구 대비 CCTV 설치 규모","(2015 대 2020년)"],
                "subtitle":["- 2015년은 푸른색, 2020년은 빨간색 원으로 표시","- 원 크기는 인구 대비 CCTV 비율",
                           "- 직선 두께와 색상은 2015년 대비 2020년 증가율"]
            },
             width=600,
             height=500
    )
    

    reg = bb.transform_regression('인구_규모_'+year,'CCTV_규모_'+year).mark_line(color=color,opacity=0.5)

    return  alt.layer(bb, reg) # 버블 차트 위에 회귀선을 겹쳐서 표시          

### 3.2 2015년도 차트 시각화

In [6]:
# 2015년도 차트를 파란색으로 시각화
bb_reg_2015 = bubble('2015', 'blue')
bb_reg_2015

alt.LayerChart(...)

### 3.3 2020년도 차트 시각화

In [7]:
# 2020년도 차트를 빨간색으로 시각화
bb_reg_2020 = bubble('2020', 'red')
bb_reg_2020

alt.LayerChart(...)

### 3.4 이동궤적 시각화

In [8]:
# 버블 이동궤적 시각화

move = alt.Chart(df).mark_line().transform_calculate(
        up_rate= '(datum.CCTV_규모_2020-datum.CCTV_규모_2015)/datum.CCTV_규모_2015'
).encode(
    alt.X('인구_규모_2015:Q',
          scale=alt.Scale(type='linear',domain=[100000,700000]),
          title='인구_규모(단위:명)'),
    alt.X2('인구_규모_2020:Q'),
    alt.Y('CCTV_규모_2015:Q',
          scale=alt.Scale(type='linear', domain=[0,7000]), #Y축 설정
          title='CCTV설치[단위:대]'),
    alt.Y2('CCTV_규모_2020:Q'),
    alt.Size('up_rate:T',legend=None),
    alt.Color('up_rate:T', legend=None),
    tooltip=[alt.Tooltip('자치구:N'),
             alt.Tooltip('up_rate:N',format='.1%',title='CCTV_증가율')]
).properties(
             width=600,
             height=500
    )
move

alt.Chart(...)

### 3.5 시각화 결과 통합

In [9]:
# 시각화 결과 통합

layer = alt.layer(bb_reg_2015,bb_reg_2020,move)
layer

alt.LayerChart(...)

- 버블 차트 시각화 결론

 CCTV 설치 규모 차트에서 인구 규모와 CCTV 설치 규모 부분에서는 2015년에서 2020년으로 갈수록 인구 규모는 줄어드는 대신 설치 규모는 증가하는 추세를 파악할 수 있다. 이는 CCTV 설치 비율이 해가 갈수록 증가하였다는 결과를 도출해 낼 수 있다. 두 번째로 이동궤적 선에서 강북구의 궤적 선이 유독 굵은 데 이는 강북구의 인구가 해마다 감소하는 동시에 자연 경관으로 인해 제한되었던 강북구의 개발이 일부 시작되면서 cctv설치율이 증가하였기 때문이다. 이와 달리 송파구에서는 다른 추세를 보인다. 이는 송파구의 지속적인 아파트 등의 개발로 송파구의 인구가 증가하였기 때문이라고 할 수 있다.

## 4. 단계구분도 시각화

### 4.1 자치구 위치 시각화

In [10]:
# 자치구 위치 시각화
# points = alt.layer(
#             alt.Chart(                                                       #뒤에 지도 배경도 함께
#             alt.topo_feature(seoul_url, 'seoul_municipalities_geo')
#             ).mark_geoshape(
#                 fill='#ddd',stroke='#aaa', strokeWidth=0.25
#             ),
#             alt.Chart(
#                 offices
#             ).mark_circle(
#             ).encode(
#                 latitude='위도:Q',
#                 longitude='경도:Q',
#                 tooltip='자치구청:N',
#                 size=alt.value(50)
#             )
#     ).properties(
#     width=400,
#     height=300
# )

points=alt.Chart(  #위치를 점의로만 표시
            offices
).mark_circle(
            ).encode(
                latitude='위도:Q',
                longitude='경도:Q',
                tooltip='자치구청:N',
                size=alt.value(50)
            ).properties(
    width=400,
    height=300
)
points

alt.Chart(...)

### 4.2 단계구분도를 반환하는 함수 정의

In [11]:
# 자치구 cctv 비율 단계구분도 

def map(year):
    map= alt.Chart(seoul).mark_geoshape(
        stroke='#aaa', strokeWidth=0.25
    ).transform_lookup(
        lookup='properties.ESRI_PK',
        from_=alt.LookupData(data=df, key='no', fields=['자치구','CCTV_규모_'+year,'인구_규모_'+year])
    ).transform_calculate(
        rate='datum.CCTV_규모_'+year+'/datum.인구_규모_'+year
    ).encode(
        alt.Color('rate:Q',legend=None),
        tooltip=[
            '자치구:N',
            alt.Tooltip('rate:N',format='.2%',title='CCTV 비율'),
            alt.Tooltip('CCTV_규모_'+year+':Q',format=',.0f',title='CCTV 규모'),
            alt.Tooltip('인구_규모_'+year+':Q',format=',.0f',title='인구 규모')
        ]
    ).properties(
        title={
                "text":["CCTV 설치 비율"],
                "subtitle":[year+"년"]
            },
        width=400,
        height=300
    )
    
    
    return map

### 4.3 단계구분도 종합

In [12]:
map2015 = map('2015')
map2020 = map('2020')

# la= alt.layer(map2015,points)
# la
alt.hconcat(
    alt.layer(map2015,points),alt.layer(map2020,points)
 ).configure_view(
     stroke=None
 )

alt.HConcatChart(...)

- 단계구분도 시각화 결론

 CCTV 설치 비율의 단계 구분도를 보았을 때 2015년도에서의 종로구와 함께 2020년에서는 중구와 강남구에서 우세를 띈다. 이는 중구와 종로구 도심 공동화 현상으로 거주인구가 서울 외곽으로 빠져나갔기 때문에 증가율이 높다고 할 수 있다. 이에 비해 노원구 등의 거주인구가 많은 곳은 상대적으로 CCTV 설치율 증가폭이 상대적으로 더 낮게 관측된다. 또한 강남구는 지속적인 지역 개발이 요인이 강남구의 cctv 증가율 우세의 원인이 된 것이라고 말할 수 있다.  

## 5. 결론


### 배운점

 함수를 통해 차트의 공통된 속성을 인코딩하고 변수를 입력해 함수를 호출해 원하는 모양으로 차트를 그려내는 방식을 알게 되었다. 

### 느낀점

 시각화를 할 때는 불필요한 중복 작업을 최소화하고 모두의 시선에서 이해할 수 있는 자료를 만드는 점이 중요하다는 점을 느끼게 되었다. 왜냐하면 변화의 정도는 데이터 값만 봐도 분석이 될 것 같다고 지금까지 생각을 해왔기 때문이다. 하지만 데이터를 직접 시각화해 보니 이 시각화 자료가 내 관점뿐만 아닌 아무것도 모르는 타인의 관점에서 보았을때도 한 눈에 그 사람을 알기 쉽게 이해시킬 수 있을 것 같다는 생각이 들었다. 그리고 일일이 차트를 코딩하지 않고 함수를 통해 공통된 부분을 코딩하고 변수 값만 주어 원하는 모습으로 시각화 할 수 있는 효울성을 깨닫게 되었다.

### 각오

 변이율을 중점으로 살펴볼 수 있는 다른 데이터들도 시각화 해보아야 겠다.